# Phase 1: Northwind Traders 

## Pure SQL Analysis

This notebook contains SQL-only solutions for the four analysis parts. 

> Queries are executed against `data/northwind.db` for verification and Phase 1 report.

## Setup: Connect to Database

In [ ]:
import sqlite3
from pathlib import Path

# Resolve path whether notebook is run from project root or notebooks/
db_path = next((p for base in [Path.cwd(), Path.cwd().parent] for p in [base / "data" / "northwind.db"] if p.exists()), None)

if db_path is None:
    raise FileNotFoundError("data/northwind.db not found (run from project root or notebooks/)")
conn = sqlite3.connect(db_path.resolve()) # Open connection

Sample test query for information retrieval

In [ ]:
# Test the connection with a simple query
import pandas as pd

query_employees="""
SELECT
    e.FirstName || ' ' || e.LastName AS EmployeeName,
    e.BirthDate,
    e.HireDate,
    e.City,
    e.Country

from Employees e;
"""


data_employees = pd.read_sql_query(query_employees, conn)
data_employees.sample(n=5)

In [ ]:
query_join = """
SELECT
    e.EmployeeID,
    e.FirstName || ' ' || e.LastName AS EmployeeName,
    o.OrderID,
    o.OrderDate,
    ROUND(SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)), 2) AS TotalSales
FROM Employees e
JOIN Orders o ON e.EmployeeID = o.EmployeeID
JOIN "Order Details" od ON o.OrderID = od.OrderID
GROUP BY e.EmployeeID, e.FirstName, e.LastName
"""

order_data = pd.read_sql_query(query_join, conn)
order_data.sample(n=5)

## Part 1: Employee Sales Performance Analysis

**Objective:** Identify top-performing employees based on their total sales.

- Calculate the total sales amount for each employee.
- Rank employees based on their total sales performance using `RANK()`.

In [ ]:
# Connection to database is available as `conn`

query_part1 = """
-- Part 1: Employee sales. Revenue = UnitPrice * Quantity * (1 - Discount).
-- RANK() leaves gaps after ties (e.g. 1,2,2,4); use when distinct rank positions matter.
SELECT
    e.EmployeeID,
    e.FirstName || ' ' || e.LastName AS EmployeeName,
    ROUND(SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)), 2) AS TotalSales,
    RANK() OVER (ORDER BY SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)) DESC) AS SalesRank
FROM Employees e
JOIN Orders o ON e.EmployeeID = o.EmployeeID
JOIN "Order Details" od ON o.OrderID = od.OrderID
GROUP BY e.EmployeeID, e.FirstName, e.LastName
ORDER BY SalesRank;
"""

import pandas as pd

df1 = pd.read_sql_query(query_part1, conn)
df1.sample(n=5)

## Part 2: Monthly Sales Trend Analysis

**Objective:** Analyze monthly sales trends to identify peak sales periods.

- Aggregate total sales for each month.
- Calculate the month-over-month sales growth using `LAG()`.

In [ ]:
query_part2 = """
-- Part 2: Monthly trend + MoM growth. CTE keeps aggregation and window logic clear.
WITH MonthlySales AS (
    -- strftime in SQLite buckets OrderDate into YYYY-MM for monthly rollup
    SELECT
        strftime('%Y-%m', o.OrderDate) AS Month,
        SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)) AS TotalSales
    FROM Orders o
    JOIN "Order Details" od ON o.OrderID = od.OrderID
    GROUP BY strftime('%Y-%m', o.OrderDate)
)
SELECT
    Month,
    ROUND(TotalSales, 2) AS TotalSales,
    -- LAG = previous row's TotalSales; NULLIF avoids divide-by-zero on first month
    ROUND(LAG(TotalSales) OVER (ORDER BY Month), 2) AS PrevMonthSales,
    ROUND((TotalSales - LAG(TotalSales) OVER (ORDER BY Month)) /
          NULLIF(LAG(TotalSales) OVER (ORDER BY Month), 0) * 100, 2) AS MoM_Growth_Pct
FROM MonthlySales
ORDER BY Month;
"""

df2 = pd.read_sql_query(query_part2, conn)
df2.sample(n=5)

## Part 3: Product Sales Ranking by Category

**Objective:** Determine the top-selling products within each category.

- Calculate total sales for each product.
- Rank products within their categories using `RANK()`.

In [ ]:
from random import random


query_part3 = """
-- Part 3: Top products per category. PARTITION BY resets rank within each category.
SELECT
    c.CategoryName,
    p.ProductName,
    ROUND(SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)), 2) AS TotalSales,
    RANK() OVER (PARTITION BY c.CategoryID ORDER BY SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)) DESC) AS CategoryRank
FROM Categories c
JOIN Products p ON c.CategoryID = p.CategoryID
JOIN "Order Details" od ON p.ProductID = od.ProductID
GROUP BY c.CategoryID, c.CategoryName, p.ProductID, p.ProductName
ORDER BY c.CategoryName, CategoryRank;
"""

df3 = pd.read_sql_query(query_part3, conn)
df3.sample(n=5)

## Part 4: Customer Purchase Behavior Analysis

**Objective:** Identify customers with the highest average order value.

- Calculate the average order value for each customer.
- Rank customers by average order value. Show customers with Ranks 2, 3, 5, 8, 12, 15, and 17 using `RANK()`.

In [ ]:
query_part4 = """
-- Part 4: Customer AOV. Two CTEs: per-order totals, then per-customer AOV + global rank.
WITH OrderTotals AS (
    -- One row per order: total revenue (needed before AVG per customer)
    SELECT OrderID, SUM(UnitPrice * Quantity * (1 - Discount)) AS OrderTotal
    FROM "Order Details"
    GROUP BY OrderID
),
CustomerAOV AS (
    SELECT
        c.CustomerID,
        c.CompanyName,
        ROUND(AVG(ot.OrderTotal), 2) AS AvgOrderValue,
        RANK() OVER (ORDER BY AVG(ot.OrderTotal) DESC) AS AOV_Rank
    FROM Customers c
    JOIN Orders o ON c.CustomerID = o.CustomerID
    JOIN OrderTotals ot ON o.OrderID = ot.OrderID
    GROUP BY c.CustomerID, c.CompanyName
)
-- Assignment asks for specific ranks to verify RANK() behavior (gaps after ties)
SELECT * FROM CustomerAOV WHERE AOV_Rank IN (2, 3, 5, 8, 12, 15, 17)
ORDER BY AOV_Rank;
"""

df4 = pd.read_sql_query(query_part4, conn)
df4.sample(n=5)

In [ ]:
conn.close()